In [1]:
import requests

In [2]:
#导入模块
import urllib2
import re
import os
import glob

#设定抓取页数
page_amount = 10

#抓取首页的html代码   
def get_page(url):
    req = urllib2.Request(url)
    req.add_header('User-Agent','Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.111 Safari/537.36')      #缺省部分填上浏览器字符串
    response = urllib2.urlopen(req)
    html = response.read().decode('utf-8')
    return html

#抓取图片
def read_image(url):
    req = urllib2.Request(url)
    req.add_header('User-Agent','Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/40.0.2214.111 Safari/537.36')      #缺省部分填上浏览器字符串
    response = urllib2.urlopen(url)
    html = response.read()
    return html

#得到当前的最新页面数，从这个页面开始倒着爬，因为用了这个脚本以后以前的图可能已经看过了
def get_current_page_number(html):
    match = re.search(r'<span class="current-comment-page">\[(.*)\]</span>',html)
    return match.group(1)

#得到图片列表
def get_picturs_url_list(url):
    html = get_page(url)
    l = re.findall(r'<p><img src="http://.*.sinaimg.cn/mw600/.*jpg" /></p>',html)
    result = []
    for string in l:
        src = re.search(r'"(.*)"',string)
        result.append(str(src.group(1)))    #解决Unicode编码开头问题，有空好好补下编码和字符规范
    return result

#下载图片并存储到本地文件夹
def image_save(url,number):
    number = str(number)
    print '正在抓取第',number,'张'
    filename = number + '.jpg'
    with open(filename,'wb') as fp:
        img = read_image(url)
        fp.write(img)

#准备存放图片的文件夹，并进入到指定路径

os.chdir('/Users/brettlv/')
os.mkdir('pic6')
os.chdir(os.path.join(os.getcwd(), 'pic6'))    

#主函数
def main1():
    html = get_page('http://jandan.net/ooxx')
    number = int(get_current_page_number(html))
    l = []
    amount = 0
    for n in range(0,page_amount):
        url = 'http://jandan.net/ooxx/page-' + str(number-n) + '#comments'
        l += get_picturs_url_list(url) 
    
    for url in l:
        amount += 1
        image_save(url,amount)
main1()

OSError: [Errno 17] File exists: 'pic6'

In [3]:
import requests, re, os

url = 'http://tieba.baidu.com/p/2166231880'

header = {
        'Accept': '*/*',
        'Accept-Encoding':'gzip,deflate,sdch',
        'Accept-Language':'zh-CN,zh;q=0.8',
        'Connection':'keep-alive'
        }
html = requests.get(url,headers = header)

data = html.content.decode('utf-8')
find = re.compile(r'<img pic_type="0" class="BDE_Image" src="(.*?).jpg" bdwater')
result = find.findall(data)
os.chdir('/Users/brettlv/')
os.mkdir('pic4')
os.chdir(os.path.join(os.getcwd(), 'pic4'))

for img_url in result:
    name = img_url.split('/')[-1]
    img_url = img_url+'.jpg'
    html = requests.get(img_url,headers = header)
    im = html.content
    with open(name+'.jpg','wb')as f:
        f.write(im)

OSError: [Errno 17] File exists: 'pic4'

In [6]:
import urllib2
from HTMLParser import HTMLParser
from traceback import print_exc
from sys import stderr

class _DeHTMLParser(HTMLParser):
    '''
    利用HTMLParse来解析网页元素
    '''
    def __init__(self):
        HTMLParser.__init__(self)
        self.img_links = []

    def handle_starttag(self, tag, attrs):
        if tag == 'img':
            # print(attrs)
            try:
                if ('pic_type','0') in attrs:
                    for name, value in attrs:
                        if name == 'src':
                            self.img_links.append(value)
            except Exception as e:
                print(e)


        return self.img_links


def dehtml(text):
    try:
        parser = _DeHTMLParser()
        parser.feed(text)
        parser.close()
        return parser.img_links
    except:
        print_exc(file=stderr)
        return text

os.chdir('/Users/brettlv/')
os.mkdir('pic7')
os.chdir(os.path.join(os.getcwd(), 'pic7'))

def main2():
    html = urllib2.urlopen('http://tieba.baidu.com/p/2166231880')
    content = html.read()
    #print(dehtml(content))
    i = 0
    for img_list in dehtml(content):
        img_content = urllib2.urlopen(img_list).read()
        path_name = str(i)+'.jpg'
        with open(path_name,'wb') as f:
            f.write(img_content)
        i+=1
main2()

Traceback (most recent call last):
  File "<ipython-input-6-1cc176fc7daf>", line 32, in dehtml
    parser.feed(text)
  File "/Users/brettlv/anaconda3/envs/ipykernel_py2/lib/python2.7/HTMLParser.py", line 117, in feed
    self.goahead(0)
  File "/Users/brettlv/anaconda3/envs/ipykernel_py2/lib/python2.7/HTMLParser.py", line 161, in goahead
    k = self.parse_starttag(i)
  File "/Users/brettlv/anaconda3/envs/ipykernel_py2/lib/python2.7/HTMLParser.py", line 308, in parse_starttag
    attrvalue = self.unescape(attrvalue)
  File "/Users/brettlv/anaconda3/envs/ipykernel_py2/lib/python2.7/HTMLParser.py", line 475, in unescape
    return re.sub(r"&(#?[xX]?(?:[0-9a-fA-F]+|\w{1,8}));", replaceEntities, s)
  File "/Users/brettlv/anaconda3/envs/ipykernel_py2/lib/python2.7/re.py", line 155, in sub
    return _compile(pattern, flags).sub(repl, string, count)
UnicodeDecodeError: 'ascii' codec can't decode byte 0xe7 in position 28: ordinal not in range(128)


ValueError: unknown url type: <